In [1]:
 using Plots, BenchmarkTools


In [ ]:
struct LogSpacingRange{T}
    start::T
    stop::T
    base::T
    size::Int64

    function LogSpacingRange(start::Real, stop::Real, size::Int64, base::Real)
        @assert base > 1
        @assert size ≥ 1
        ftype = promote_type(typeof(start), typeof(stop))
        if ~(ftype <: AbstractFloat)
            ftype = Float64
        end

        return new{ftype}(start, stop, base, size)
    end
end

Base.size(p::LogSpacingRange) = (p.size, )
Base.length(p::LogSpacingRange) = p.size
Base.iterate(p::LogSpacingRange, state=1) = state > p.size ? nothing : (p[state], state+1)
Base.eltype(p::LogSpacingRange{T}) where {T} = T
Base.IteratorSize(::LogSpacingRange{T}) where {T} = Base.HasLength()
Base.IteratorEltype(::LogSpacingRange{T}) where {T} = Base.HasEltype()
Base.firstindex(p::LogSpacingRange) = (p.base)^(p.start)
Base.lastindex(p::LogSpacingRange) = (p.base)^(p.stop)
function Base.getindex(p::LogSpacingRange, i::Int64) 
    @assert 0 < i ≤ p.size
    if p.size == 1
        return p.start
    else 
        r = p.start + (p.stop-p.start)/(p.size-1)*(i-1)
        return (p.base)^r
    end
end

function logspace(a, b, n::Integer=10, base::Real=10)
    return LogSpacingRange(a, b, n, base)
end



In [ ]:
for v in logspace(1, 5, 3, 10)
    @show v
end 

In [ ]:
q = logspace(1, 5, 3, 10)


In [ ]:
lastindex(q)

In [ ]:
# @btime begin
#     a=logspace(1, 100, 1000_0000)
#     sin.(a)
# end

In [ ]:
# @btime begin
#     a=10.0.^range(1.0, 100.0, length=1000_0000)
#     sin.(a)
# end

In [72]:
struct ShiftedIndexedArray{T, N, AA <: AbstractArray{T, N}} 
    A::AA
    offsets::NTuple{N, Int}
    
    function ShiftedIndexedArray{T, N, AA}(arr::AA, offsets::NTuple{N, Int}) where {T, N, AA<:AbstractArray{T, N}}
        return new{T, N, AA}(arr, offsets)
    end
end

function ShiftedIndexedArray(arr::AA, offsets::NTuple{N, Int}) where {N, AA<:AbstractArray}
    @assert ndims(arr) == N
    return ShiftedIndexedArray{eltype(arr), ndims(arr), typeof(arr)}(arr, offsets)
end

function ShiftedIndexedArray(arr::AA, offsets::Integer) where {AA<:AbstractVector}
    return ShiftedIndexedArray{eltype(arr), 1, typeof(arr)}(arr, (offsets, ))
end

const ShiftedIndexedVector{T, AA<: AbstractVector{T}} = ShiftedIndexedArray{T, 1, AA}
const ShiftedIndexedMatrix{T, AA<:AbstractMatrix{T}} = ShiftedIndexedArray{T, 2, AA}

ShiftedIndexedVector(vec::AbstractVector{T}, offset::Integer) where {T}= ShiftedIndexedArray(vec, offset)
ShiftedIndexedMatrix(mat::AbstractMatrix{T}, offsets::NTuple{2, Int}) where {T} = ShiftedIndexedArray(mat, offsets)

function _original_cartesian_index(S::ShiftedIndexedArray{T, N, AA}, ind::NTuple{N, Int}) where {T, N, AA<:AbstractArray{T, N}}
    return ind .- S.offsets .+ ntuple(d->1, N)
end

function _original_linear_index(S::ShiftedIndexedArray{T, N, AA}, ind::Integer) where {T, N, AA<:AbstractArray{T, N}}
    return ind - prod(S.offsets) + 1
end


Base.size(S::ShiftedIndexedArray) = size(S.A)

Base.length(S::ShiftedIndexedArray) = length(S.A)
Base.firstindex(S::ShiftedIndexedArray) = firstindex(S.A)
Base.lastindex(S::ShiftedIndexedArray) = lastindex(S.A)
# Base.getindex(S::ShiftedIndexedArray, i::Int) = getindex(S.A, i)
original(S::ShiftedIndexedArray) = S.A




original (generic function with 1 method)

In [73]:
B=ShiftedIndexedArray(rand(3,3), (-4, -1))

ShiftedIndexedArray{Float64, 2, Matrix{Float64}}([0.9871723764338934 0.8030155674686781 0.07121566341709762; 0.26737587704516685 0.19248131847314054 0.9829529557857537; 0.027456905923424202 0.8649441041072908 0.5093480981545042], (-4, -1))

In [74]:
function Base.getindex(S::ShiftedIndexedArray{T, N, AA}, I...) where {T, N, AA}
    oind = _original_cartesian_index(S, I)
    if checkbounds(Bool, S.A, oind...) 
        return getindex(S.A, oind...)
    else
        throw(BoundsError(S, I...))
    end
end

function Base.getindex(S::ShiftedIndexedArray{T, N, AA}, i::Int64) where {T, N, AA}
    oind = _original_linear_index(S, i)
    if checkbounds(Bool, S.A, oind)
        return getindex(S.A, oind)
    else 
        throw(BoundsError(S, i))
    end
end

function Base.setindex!(S::ShiftedIndexedArray{T, N, AA}, v, I... ) where {T, N, AA}
    oind = _original_cartesian_index(S, I)
    if checkbounds(Bool, S.A, oind...) == false
        throw(BoundsError(S, I...))
    else
        return setindex!(S.A, v, oind...)
    end
end

function Base.setindex!(S::ShiftedIndexedArray{T, N, AA}, v, i::Int ) where {T, N, AA}
    oind = _original_liear_index(S, i)
    if checkbounds(Bool, S.A, oind) == false
        throw(BoundsError(S, i))
    else
        return setindex!(S.A, v, oind)
    end
end


Base.similar(S::ShiftedIndexedArray) = ShiftedIndexedArray(Base.similar(S.A), S.offsets)
Base.sizeof(S::ShiftedIndexedArray) = sizeof(S.A)

Base.iterate(S::ShiftedIndexedArray, state=1) = iterate(S.A, state)
Base.IndexStyle(S::ShiftedIndexedArray) = IndexStyle(S.A)

# Base.keys(S::ShiftedIndexedArray) = prod(S.offsets):(prod(S.offsets) + length(S)-1)

In [75]:
B = [3,4,5,6]
A = ShiftedIndexedArray(B, -3)

ShiftedIndexedVector{Int64, Vector{Int64}}([3, 4, 5, 6], (-3,))

In [76]:
for v in eachindex(A)
    @show A[v]
end

A[v] = 3
A[v] = 4
A[v] = 5
A[v] = 6


In [77]:
X=ShiftedIndexedArray(reshape(collect(1:12), (3, 4)), (-3, -4))


ShiftedIndexedArray{Int64, 2, Matrix{Int64}}([1 4 7 10; 2 5 8 11; 3 6 9 12], (-3, -4))

In [65]:
X[-3, -3]

4

In [66]:
X.A

3×4 Matrix{Int64}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [79]:
collect(X)

12-element Vector{Any}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [ ]:
IndexStyle(A)

In [ ]:
B

In [ ]:
eachindex(A)

In [ ]:
Base.OneTo(length(B))

In [ ]:
eachindex(A)

In [ ]:
eachindex(A)

In [ ]:
eachindex(B)

In [ ]:
B[-5]